##### The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".
https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [1]:
#importing Required Libraries
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
#Reading dataset
df_jan23 = pd.read_parquet("D:\Datascience\MLOPS_Zoomcamp\yellow_tripdata_2023-01.parquet")
df_feb23 = pd.read_parquet("D:\Datascience\MLOPS_Zoomcamp\yellow_tripdata_2023-02.parquet")

In [3]:
#inspect Data
df_jan23.sample(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
546817,1,2023-01-07 12:16:11,2023-01-07 12:21:28,2.0,2.40,1.0,N,137,232,1,11.4,2.5,0.5,1.00,0.0,1.0,16.40,2.5,0.0
2579369,2,2023-01-27 17:45:39,2023-01-27 18:02:30,1.0,6.31,1.0,N,244,142,2,27.5,2.5,0.5,0.00,0.0,1.0,34.00,2.5,0.0
1105846,2,2023-01-13 08:50:29,2023-01-13 08:57:27,1.0,0.96,1.0,N,246,234,1,7.9,0.0,0.5,2.38,0.0,1.0,14.28,2.5,0.0
1203843,2,2023-01-14 00:24:10,2023-01-14 00:36:32,4.0,1.02,1.0,N,79,144,1,12.1,1.0,0.5,1.00,0.0,1.0,18.10,2.5,0.0
44097,2,2023-01-01 15:03:04,2023-01-01 15:18:36,5.0,2.97,1.0,N,236,48,2,18.4,0.0,0.5,0.00,0.0,1.0,22.40,2.5,0.0


In [4]:
df_jan23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [5]:
df_jan23.describe()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
count,3.066766e+06,2.995023e+06,3.066766e+06,2.995023e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,3.066766e+06,2.995023e+06,2.995023e+06
mean,1.730215e+00,1.362532e+00,3.847342e+00,1.497440e+00,1.663980e+02,1.643926e+02,1.194483e+00,1.836707e+01,1.537842e+00,4.882900e-01,3.367941e+00,5.184907e-01,9.820847e-01,2.702038e+01,2.274231e+00,1.074086e-01
std,4.438480e-01,8.961200e-01,2.495838e+02,6.474767e+00,6.424413e+01,6.994368e+01,5.294762e-01,1.780782e+01,1.789592e+00,1.034641e-01,3.826759e+00,2.017579e+00,1.833529e-01,2.216359e+01,7.718454e-01,3.556511e-01
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-9.000000e+02,-7.500000e+00,-5.000000e-01,-9.622000e+01,-6.500000e+01,-1.000000e+00,-7.510000e+02,-2.500000e+00,-1.250000e+00
25%,1.000000e+00,1.000000e+00,1.060000e+00,1.000000e+00,1.320000e+02,1.140000e+02,1.000000e+00,8.600000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,1.000000e+00,1.540000e+01,2.500000e+00,0.000000e+00
50%,2.000000e+00,1.000000e+00,1.800000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,1.280000e+01,1.000000e+00,5.000000e-01,2.720000e+00,0.000000e+00,1.000000e+00,2.016000e+01,2.500000e+00,0.000000e+00
75%,2.000000e+00,1.000000e+00,3.330000e+00,1.000000e+00,2.340000e+02,2.340000e+02,1.000000e+00,2.050000e+01,2.500000e+00,5.000000e-01,4.200000e+00,0.000000e+00,1.000000e+00,2.870000e+01,2.500000e+00,0.000000e+00
max,2.000000e+00,9.000000e+00,2.589281e+05,9.900000e+01,2.650000e+02,2.650000e+02,4.000000e+00,1.160100e+03,1.250000e+01,5.316000e+01,3.808000e+02,1.969900e+02,1.000000e+00,1.169400e+03,2.500000e+00,1.250000e+00


In [6]:
df_jan23.shape

(3066766, 19)

In [7]:
df_jan23.shape[1]

19

Q2. Computing duration
- Now let's compute the duration variable. It should contain the duration of a ride in minutes.

- What's the standard deviation of the trips duration in January?

"""We need to create a column called `duration` 
that is derived from the `tpep_pickup_datetime` and `tpep_dropoff_datetime"""


In [8]:
df_jan23['duration']=df_jan23['tpep_dropoff_datetime']-df_jan23['tpep_pickup_datetime']
df_jan23.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,0 days 00:08:26
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,0 days 00:06:19
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,0 days 00:12:45
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,0 days 00:09:37
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,0 days 00:10:50


Note: There are 0 days which is being shown. We need the duration in minutes so that it becomes an easy problem to frame for machine learning. In order to achive this, we will convert the values in duration column to seconds and then divide by 60 to get the duration in minutes.

In [9]:
df_jan23['duration']=df_jan23['duration'].apply(lambda x:x.total_seconds()/60)


In [10]:
df_jan23.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [11]:
df_jan23.std()

C:\Users\Lolitha\AppData\Local\Temp\ipykernel_24120\1809543087.py:1: FutureWarning: The default value of numeric_only in DataFrame.std is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_jan23.std()


VendorID                                  0.443848
tpep_pickup_datetime     9 days 15:29:51.002042903
tpep_dropoff_datetime    9 days 15:29:27.269195801
passenger_count                            0.89612
trip_distance                           249.583756
RatecodeID                                6.474767
PULocationID                             64.244131
DOLocationID                             69.943682
payment_type                              0.529476
fare_amount                              17.807822
extra                                     1.789592
mta_tax                                   0.103464
tip_amount                                3.826759
tolls_amount                              2.017579
improvement_surcharge                     0.183353
total_amount                             22.163589
congestion_surcharge                      0.771845
airport_fee                               0.355651
duration                                 42.594351
dtype: object

Q3. Dropping outliers
- Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

- What fraction of the records left after you dropped the outliers?

In [12]:
df_jan23['duration'].describe()

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
Name: duration, dtype: float64

In [13]:
# with the above data, it makes sense to instead look at the higher and lower percentiles
df_jan23['duration'].describe(percentiles=[0.10, 0.95, 0.99])

count    3.066766e+06
mean     1.566900e+01
std      4.259435e+01
min     -2.920000e+01
10%      4.483333e+00
50%      1.151667e+01
95%      3.646667e+01
99%      5.725000e+01
max      1.002918e+04
Name: duration, dtype: float64

Observation: Here we can see that the lowest and highest duration are clearly some outliers. So we will only focus on the trips that were more than 1 minute and less than 60 minutes.


In [14]:
df_jan23_filtered= df_jan23[(df_jan23['duration'] >= 1) & (df_jan23['duration'] < 60)]
df_jan23_filtered.shape[0]

3009130

In [15]:
#Calculating fraction of the datapoints after outlier removalfrom the original data
df_jan23_filtered.shape[0]/df_jan23.shape[0]

0.9812062609276352

Q4. Dimentionality after One-hot encoding
- Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

- Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
- Fit a dictionary vectorizer
- Get a feature matrix from it
- What's the dimensionality of this matrix (number of columns)?

In [16]:
"""In this part, we will perform feature selection.
We will only consider features that might have an impact on the duration."""
df_jan23_filtered.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [17]:
df_jan23_filtered.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'duration'],
      dtype='object')

In [18]:
categorical = ['PULocationID', 'DOLocationID']
numerical =['trip_distance']
#converting int to str
df_jan23_filtered[categorical] = df_jan23_filtered[categorical].astype(str)

C:\Users\Lolitha\AppData\Local\Temp\ipykernel_24120\3342304867.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_jan23_filtered[categorical] = df_jan23_filtered[categorical].astype(str)


In [19]:
#Creating our training data as a dictionary which comprises of the above categorical and numerical data
train_dict = df_jan23_filtered[categorical+ numerical].to_dict(orient='records')
dv= DictVectorizer()
X_train =dv.fit_transform(train_dict)
#Get the dimensionality of the feature matrix
dimensionality=X_train.shape[1]
print("Dimensionality of the feature matrix:", dimensionality)
#Creating y-Train
target='duration'
y_train=df_jan23_filtered['duration'].values                      

Dimensionality of the feature matrix: 516


In [20]:
#dimensionality
X_train.shape

(3009130, 516)

In [21]:
len(y_train)

3009130

Q5. Training a model
- Now let's use the feature matrix from the previous step to train a model.

- Train a plain linear regression model with default parameters
- Calculate the RMSE of the model on the training data
- What's the RMSE on train?

In [22]:
#Trainingthe base line model
lr= LinearRegression()
lr.fit(X_train, y_train)
#Using our traied linear regression model to get predictions on thir training data
y_predictions=lr.predict(X_train)

#Calculate the RMSE on the training data
rmse_train= mean_squared_error(y_train,y_predictions, squared=False)
print("RMSE on  training data:", rmse_train)

RMSE on  training data: 7.657684572051222


Q6. Evaluating the model
- Now let's apply this model to the validation dataset (February 2023).

- What's the RMSE on validation?

In [24]:
df_feb23.sample(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
211022,2,2023-02-02 23:35:15,2023-02-02 23:49:11,1.0,3.82,1.0,N,236,113,1,17.7,1.0,0.5,5.68,0.00,1.0,28.38,2.5,0.00,13.933333
135704,2,2023-02-02 11:39:17,2023-02-02 11:48:30,1.0,0.93,1.0,N,48,162,1,10.0,0.0,0.5,2.80,0.00,1.0,16.80,2.5,0.00,9.216667
2144674,2,2023-02-22 11:58:11,2023-02-22 12:05:44,1.0,0.66,1.0,N,163,162,1,8.6,0.0,0.5,2.52,0.00,1.0,15.12,2.5,0.00,7.550000
2388356,2,2023-02-24 15:10:17,2023-02-24 15:45:40,1.0,9.33,1.0,N,138,229,1,44.3,5.0,0.5,7.00,6.55,1.0,68.10,2.5,1.25,35.383333
2730147,2,2023-02-27 21:02:52,2023-02-27 21:11:05,1.0,1.35,1.0,N,237,170,1,10.0,1.0,0.5,3.75,0.00,1.0,18.75,2.5,0.00,8.216667


In [25]:
df_feb23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2913955 entries, 0 to 2913954
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [26]:
df_feb23.describe()

,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration
count,2.913955e+06,2.837138e+06,2.913955e+06,2.837138e+06,2.913955e+06,2.913955e+06,2.913955e+06,2.913955e+06,2.913955e+06,2.913955e+06,2.913955e+06,2.913955e+06,2.913955e+06,2.913955e+06,2.837138e+06,2.837138e+06,2.913955e+06
mean,1.731282e+00,1.352847e+00,3.868058e+00,1.514554e+00,1.662570e+02,1.642317e+02,1.182896e+00,1.822038e+01,1.560472e+00,4.878752e-01,3.384825e+00,5.113983e-01,9.813052e-01,2.689848e+01,2.285299e+00,9.677754e-02,1.601591e+01
std,4.524952e-01,8.891265e-01,2.684273e+02,6.633395e+00,6.413230e+01,6.975406e+01,5.297219e-01,1.749819e+01,1.792708e+00,1.006192e-01,3.838069e+00,2.024488e+00,1.873103e-01,2.184110e+01,7.609081e-01,3.388268e-01,4.284210e+01
min,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,-7.540500e+02,-7.500000e+00,-5.000000e-01,-9.300000e+01,-5.700000e+01,-1.000000e+00,-7.575500e+02,-2.500000e+00,-1.250000e+00,-4.361667e+01
25%,1.000000e+00,1.000000e+00,1.060000e+00,1.000000e+00,1.320000e+02,1.130000e+02,1.000000e+00,8.600000e+00,0.000000e+00,5.000000e-01,1.000000e+00,0.000000e+00,1.000000e+00,1.548000e+01,2.500000e+00,0.000000e+00,7.250000e+00
50%,2.000000e+00,1.000000e+00,1.790000e+00,1.000000e+00,1.620000e+02,1.620000e+02,1.000000e+00,1.280000e+01,1.000000e+00,5.000000e-01,2.800000e+00,0.000000e+00,1.000000e+00,2.020000e+01,2.500000e+00,0.000000e+00,1.180000e+01
75%,2.000000e+00,1.000000e+00,3.300000e+00,1.000000e+00,2.340000e+02,2.340000e+02,1.000000e+00,2.030000e+01,2.500000e+00,5.000000e-01,4.200000e+00,0.000000e+00,1.000000e+00,2.870000e+01,2.500000e+00,0.000000e+00,1.876667e+01
max,6.000000e+00,9.000000e+00,3.350043e+05,9.900000e+01,2.650000e+02,2.650000e+02,4.000000e+00,2.203100e+03,2.080000e+01,4.000000e+00,4.829000e+02,1.099000e+02,1.000000e+00,2.208100e+03,2.750000e+00,1.250000e+00,7.053617e+03


In [27]:
df_feb23.shape

(2913955, 20)

In [23]:
#preparing the validation dataset i.e df_feb23
df_feb23['duration']= df_feb23['tpep_dropoff_datetime']-df_feb23['tpep_pickup_datetime']
df_feb23['duration']=df_feb23['duration'].apply(lambda x:x.total_seconds()/60)

df_feb23_filtered =df_feb23[(df_feb23['duration']>=1) & (df_feb23['duration']<60)]
categorical=['PULocationID','DOLocationID']
numerical=['trip_distance']

#Converting to str
df_feb23_filtered[categorical]= df_feb23_filtered[categorical].astype(str)

#Creating our training data as a dictionary which comprisses of the above categorical and numerical data
val_dict=df_feb23_filtered[categorical+numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

#Creating y train
target = 'duration'
y_val = df_feb23_filtered[target].values

y_pred = lr.predict(X_val)

#Calculate the RMSE on the training data
rmse_feb_train= mean_squared_error(y_val,y_pred,squared= False)
print("RMSE on  training data:", rmse_feb_train)



C:\Users\Lolitha\AppData\Local\Temp\ipykernel_24120\1507396810.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_feb23_filtered[categorical]= df_feb23_filtered[categorical].astype(str)


RMSE on  training data: 7.818228971202427
